# Example of KEO Taylor series expansion for methane

Here we use the following internal coordinates:

1. $r_1$
2. $r_2$
3. $r_3$
4. $r_4$
5. $s_1 = (2\alpha_{12} - \alpha_{13} - \alpha_{14} - \alpha_{23} - \alpha_{24} + 2\alpha_{34})/\sqrt{12}$
6. $s_2=\alpha_{13} - \alpha_{14} - \alpha_{23} + \alpha_{24}) / 2$
7. $s_3=(\alpha_{24} - \alpha_{13}) / \sqrt{2}$
8. $s_4=(\alpha_{23} - \alpha_{14}) / \sqrt{2}$
9. $s_5=(\alpha_{34} - \alpha_{12}) / \sqrt{2}$,

where $r_i\equiv\text{C-H}_i$ and $\alpha_{ij}\equiv\text{H}_i\text{-C-H}_j$

In [1]:
import itertools

import jax
import jax.numpy as jnp
import numpy as np
from scipy.linalg import expm
from scipy.special import factorial

from vibrojet.eckart import EckartMethod, eckart
from vibrojet.jet_prim import acos, inv
from vibrojet.keo import Gmat, G_to_invcm, com
from vibrojet.taylor import deriv_list

jax.config.update("jax_enable_x64", True)

Define a function `from_symmetrized_alpha_to_alpha` to obtain five (six) $\alpha_{ij}$ valence angular coordinates from the five symmetrized $s_k$ coordinates, defined above.

In [2]:
def from_symmetrized_alpha_to_alpha(symm_alpha, alpha_ini, no_iter: int = 10):

    def symmetrized_alpha(alpha):
        alpha12, alpha13, alpha14, alpha23, alpha24 = alpha

        cosbeta = (jnp.cos(alpha23) - jnp.cos(alpha12) * jnp.cos(alpha13)) / (
            jnp.sin(alpha12) * jnp.sin(alpha13)
        )
        beta312 = acos(cosbeta)

        cosbeta = (jnp.cos(alpha24) - jnp.cos(alpha12) * jnp.cos(alpha14)) / (
            jnp.sin(alpha12) * jnp.sin(alpha14)
        )
        beta412 = acos(cosbeta)

        cosa34 = jnp.cos(alpha13) * jnp.cos(alpha14) + jnp.cos(
            beta312 + beta412
        ) * jnp.sin(alpha13) * jnp.sin(alpha14)
        alpha34 = acos(cosa34)

        sym = jnp.array(
            [
                (2 * alpha12 - alpha13 - alpha14 - alpha23 - alpha24 + 2 * alpha34)
                / jnp.sqrt(12),
                (alpha13 - alpha14 - alpha23 + alpha24) / 2,
                (alpha24 - alpha13) / jnp.sqrt(2),
                (alpha23 - alpha14) / jnp.sqrt(2),
                (alpha34 - alpha12) / jnp.sqrt(2),
            ]
        )
        return sym, alpha34

    alpha = alpha_ini

    for _ in range(no_iter):
        sym, alpha34 = symmetrized_alpha(alpha)
        jac = jax.jacrev(symmetrized_alpha)(alpha)[0]
        am = jac.T @ jac
        ai = inv(am)
        bm = (symm_alpha - sym) @ jac
        alpha = alpha + ai @ bm

    stdev = jnp.sqrt(jnp.mean(jnp.square(symm_alpha - sym)))

    return alpha, alpha34, stdev

Define a function that computes the rotation matrix based on the equilibrium geometry, aligning it with a reference frame that better reflects the $T_d$ molecular symmetry. This frame rotation may not be necessary in general, it is essential for comparisons with the KEO expansion results obtained from TROVE.

In [3]:
def rotation_matrix_td_frame(xyz, no_iter: int = 20):
    norm = np.linalg.norm(xyz, axis=-1)
    assert all(np.abs(norm[1:] - norm[1]) < 1e-14), f"C-H distances must all be equal"
    xyz_norm = xyz / norm[1]

    # reference frame orientation for Td-symmetry molecule
    sqrt3 = 1 / np.sqrt(3)
    xyz0 = np.array(
        [
            [0.0, 0.0, 0.0],
            [-sqrt3, sqrt3, sqrt3],
            [-sqrt3, -sqrt3, -sqrt3],
            [sqrt3, sqrt3, -sqrt3],
            [sqrt3, -sqrt3, sqrt3],
        ]
    )

    rotmat = np.eye(3)
    kappa = np.zeros((3, 3))
    zeros = np.zeros(len(xyz))

    for _ in range(no_iter):
        exp_kappa = expm(kappa).T
        rotmat = exp_kappa @ rotmat
        lmat = exp_kappa + kappa
        xyz_norm = xyz_norm @ exp_kappa.T
        amat = np.concatenate(
            (
                [xyz_norm[:, 1], xyz_norm[:, 2], zeros],
                [-xyz_norm[:, 0], zeros, xyz_norm[:, 2]],
                [zeros, -xyz_norm[:, 0], -xyz_norm[:, 1]],
            ),
            axis=-1,
        ).T
        tmat = xyz_norm @ lmat.T - xyz0
        bvec = np.concatenate((tmat[:, 0], tmat[:, 1], tmat[:, 2]), axis=0).T
        v, *_ = np.linalg.lstsq(amat, bvec, rcond=None)
        kappa = np.array(
            [
                [0.0, v[0], v[1]],
                [-v[0], 0.0, v[2]],
                [-v[1], -v[2], 0.0],
            ]
        )
    return rotmat

Define a function to compute the Cartesian coordinates of atoms from given symmetry-adapted internal coordinates.

In [4]:
masses = [
    12.0,
    1.007825035,
    1.007825035,
    1.007825035,
    1.007825035,
]  # masses of C, H1, H2, H3, H4

# reference values of internal coordinates
r_ref = 1.0859638364
q0 = np.array([r_ref, r_ref, r_ref, r_ref, 0.0, 0.0, 0.0, 0.0, 0.0])

# initial rotation matrix to a reference frame
rotmat = np.eye(3)


# @com(masses) # use `com` is `eckart` frame is not necessary
@eckart(
    q0, masses, method=EckartMethod.exp_kappa)
def internal_to_cartesian(internal_coords):
    r1, r2, r3, r4 = internal_coords[:4]  # C-H distances

    # compute valence bond angles from symmetrized coordinates

    symm_alpha = internal_coords[4:]  # s_1, ... s_5
    alpha_ini = np.array([109.5 * np.pi / 180] * 5)
    alpha, alpha34, stdev = from_symmetrized_alpha_to_alpha(symm_alpha, alpha_ini)
    alpha12, alpha13, alpha14, alpha23, alpha24 = alpha

    # compute Cartesian coordinates

    xyz_C = jnp.array([0.0, 0.0, 0.0])

    # H1
    xyz_H1 = r1 * jnp.array([0.0, 0.0, 1.0])

    # H2
    xyz_H2 = r2 * jnp.array([jnp.sin(alpha12), 0.0, jnp.cos(alpha12)])

    # H3
    alpha = alpha13
    beta = alpha23
    v12 = xyz_H1
    v23 = xyz_H2
    n2 = v12 / jnp.linalg.norm(v12)
    n3 = jnp.cross(v23, v12)
    n3 = n3 / jnp.linalg.norm(n3)
    n1 = jnp.cross(n2, n3)
    cosa3 = jnp.sum(n2 * v23) / jnp.linalg.norm(v23)
    alpha3 = acos(cosa3)
    cosphi = (jnp.cos(beta) - jnp.cos(alpha) * jnp.cos(alpha3)) / (
        jnp.sin(alpha) * jnp.sin(alpha3)
    )
    phi = acos(cosphi)
    xyz_H3 = r3 * (
        jnp.cos(alpha) * n2
        + jnp.sin(alpha) * jnp.cos(phi) * n1
        + jnp.sin(alpha) * jnp.sin(phi) * n3
    )

    # H4
    alpha = alpha14
    beta = alpha24
    v12 = xyz_H1
    v23 = xyz_H2
    n2 = v12 / jnp.linalg.norm(v12)
    n3 = jnp.cross(v12, v23)
    n3 = n3 / jnp.linalg.norm(n3)
    n1 = jnp.cross(n3, n2)
    cosa3 = jnp.sum(n2 * v23) / jnp.linalg.norm(v23)
    alpha3 = acos(cosa3)
    cosphi = (jnp.cos(beta) - jnp.cos(alpha) * jnp.cos(alpha3)) / (
        jnp.sin(alpha) * jnp.sin(alpha3)
    )
    phi = acos(cosphi)
    xyz_H4 = r4 * (
        jnp.cos(alpha) * n2
        + jnp.sin(alpha) * jnp.cos(phi) * n1
        + jnp.sin(alpha) * jnp.sin(phi) * n3
    )

    xyz = jnp.array([xyz_C, xyz_H1, xyz_H2, xyz_H3, xyz_H4])
    return xyz @ rotmat.T

Compute reference Cartesian coordinates and rotation matrix to a reference $T_d$-symmetry-compatible frame

In [5]:
xyz = internal_to_cartesian(q0)
print("Reference internal coordinates:\n", q0)
print("Reference Cartesian coordinates:\n", xyz)

rotmat = rotation_matrix_td_frame(xyz)

xyz = internal_to_cartesian(q0)
print("Reference Cartesian coordinates in Td-symmetry-compatible frame:\n", xyz)

Reference internal coordinates:
 [1.08596384 1.08596384 1.08596384 1.08596384 0.         0.
 0.         0.         0.        ]
Reference Cartesian coordinates:
 [[ 2.92903040e-17 -1.37356992e-19  6.92534613e-18]
 [ 1.02702829e-17 -1.37356992e-19  1.08596384e+00]
 [ 1.02385652e+00 -1.37356992e-19 -3.61987945e-01]
 [-5.11928262e-01 -8.86685759e-01 -3.61987945e-01]
 [-5.11928262e-01  8.86685759e-01 -3.61987945e-01]]
Reference Cartesian coordinates in Td-symmetry-compatible frame:
 [[-3.19271547e-17 -1.01126418e-17  0.00000000e+00]
 [-6.26981513e-01  6.26981513e-01  6.26981513e-01]
 [-6.26981513e-01 -6.26981513e-01 -6.26981513e-01]
 [ 6.26981513e-01  6.26981513e-01 -6.26981513e-01]
 [ 6.26981513e-01 -6.26981513e-01  6.26981513e-01]]


Compute Taylor series expansion of G-matrix using Taylor-mode AD

In [6]:
# Generate list of multi-indices specifying the integer exponents
# for each coordinate in the Taylor series expansion
max_order = 3  # max total expansion order
deriv_ind = [
    elem
    for elem in itertools.product(*[range(0, max_order + 1) for _ in range(len(q0))])
    if sum(elem) <= max_order
]

func = lambda x: Gmat(x, masses, internal_to_cartesian)

coefs = deriv_list(func, deriv_ind, q0, if_taylor=True)

Optionally, compare with Taylor series expansion of G-matrix obtained using nested `jacfwd` calls

In [7]:
def jacfwd(x0, ind):
    f = func
    for _ in range(sum(ind)):
        f = jax.jacfwd(f)
    i = sum([(i,) * o for i, o in enumerate(ind)], start=tuple())
    return f(x0)[:, :, *i]


coefs2 = np.array([jacfwd(q0, ind) / np.prod(factorial(ind)) for ind in deriv_ind])

print("max difference for G-matrix:", np.max(np.abs(coefs - coefs2)))

2025-04-07 06:10:24.094588: E external/xla/xla/service/slow_operation_alarm.cc:73] 
********************************
[Compiling module jit_Gmat] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
2025-04-07 06:44:46.431566: E external/xla/xla/service/slow_operation_alarm.cc:140] The operation took 36m22.344026s

********************************
[Compiling module jit_Gmat] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************


max difference for G-matrix: 3.0127011996228248e-12


Optionally, compare with results of TROVE expansion

In [8]:
from zipfile import ZipFile

# Extract G-matrix Taylor series expansion coefficients from TROVE output file.

def load_trove_gmat(filename):
    deriv_ind = {}
    coefs = {}
    with ZipFile(filename, "r") as z:
        files = z.namelist()
        with z.open(files[0]) as fl:
            read_g = False
            for line in fl:
                w = line.decode("utf-8").strip().split()
                if (
                    len(w) == 6
                    and w[0] == "derivatives"
                    and w[1] == "of"
                    and w[2] == "G("
                    and w[5] == ")"
                ):
                    i = int(w[3])
                    j = int(w[4])
                    deriv_ind[(i, j)] = []
                    coefs[(i, j)] = []
                    read_g = True
                    continue
                if len(w) == 1 and w[0] == "expand_kinetic/done":
                    break
                if read_g:
                    if len(w) == 0:
                        read_g = False
                    else:
                        ind = [int(elem) for elem in w[2:11]]
                        c = float(w[12]) / np.prod(factorial(ind)) * G_to_invcm
                        deriv_ind[(i, j)].append(ind)
                        coefs[(i, j)].append(c)
    return deriv_ind, coefs


deriv_ind_trove, coefs_trove = load_trove_gmat(
    "../etc/data/methane/ch4_P14curv_fastc_36n_dip_ADF_dms.out.zip"
)

In [9]:
thresh = 1e-06

max_diff = np.zeros((15, 15))
err = []

for i in range(len(deriv_ind)):
    ind = deriv_ind[i]
    for k in range(coefs[i].shape[0]):
        for l in range(k, coefs[i].shape[1]):
            c = coefs[i, k, l]
            if np.abs(c) > thresh:
                try:
                    ind_ = deriv_ind_trove[(k + 1, l + 1)].index(list(ind))
                except ValueError:
                    raise ValueError(
                        f"can't find derivative {ind} for G({k+1},{l+1}) in TROVE output,"
                        + f"coefficient = {c}"
                    )

                c_tr = coefs_trove[(k + 1, l + 1)][ind_]
                max_diff[k, l] = max([abs(c - c_tr), max_diff[k, l]])
                # print(ind, k, l, c, c_tr, c - c_tr)

# print(max_diff)
print("max difference to TROVE:", np.max(max_diff))

max difference to TROVE: 2.9913849175500218e-12
